<a href="https://colab.research.google.com/github/wawa6609/googlecolab/blob/master/darknet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Mounting google drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My\ Drive/yolov4
#%cd darknet/

/content/drive/My Drive/yolov4


#Installing cudnn

In [6]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


In [ ]:
!mkdir cudnn

Download correct archive from nvidia page and copy it to cudnn folder. <br>
https://developer.nvidia.com/cudnn

In [12]:
!tar -xzvf cudnn/cudnn-10.1-linux-x64-v7.6.5.32.tgz -C /usr/local/
!chmod a+r /usr/local/cuda/include/cudnn.h
#!cat /usr/local/cuda/include/cudnn.h | grep CUDNN_MAJOR -A 2

mkdir: cannot create directory ‘cudnn’: File exists
cuda/include/cudnn.h
cuda/NVIDIA_SLA_cuDNN_Support.txt
cuda/lib64/libcudnn.so
cuda/lib64/libcudnn.so.7
cuda/lib64/libcudnn.so.7.6.5
cuda/lib64/libcudnn_static.a


#Downloading darknet

In [2]:
!git clone https://github.com/AlexeyAB/darknet/

Cloning into 'darknet'...
remote: Enumerating objects: 13781, done.
remote: Total 13781 (delta 0), reused 0 (delta 0), pack-reused 13781
Receiving objects: 100% (13781/13781), 12.36 MiB | 6.35 MiB/s, done.
Resolving deltas: 100% (9407/9407), done.
Checking out files: 100% (2006/2006), done.


#Installing/enabling darknet

In [ ]:
%cd darknet/
#!make
!chmod +x ./darknet

#Downloading yolo weights

In [4]:
!mkdir weights
%cd weights/

# yolov2
#!wget https://pjreddie.com/media/files/yolov2.weights

# yolov3
#!wget https://pjreddie.com/media/files/yolov3.weights

# yolov3-tiny
#!wget https://pjreddie.com/media/files/yolov3-tiny.weights

# pretrained weights
!wget https://pjreddie.com/media/files/darknet53.conv.74

%cd ..

/content/drive/My Drive/yolov4/darknet/weights
--2020-07-04 20:15:07--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  3.92MB/s    in 49s     

2020-07-04 20:15:57 (3.18 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]

/content/drive/My Drive/yolov4/darknet


#Own data for net training

In [ ]:
!mkdir ../MyNet/

Copy dataset to MyNet folder.

In [5]:
!cp -r ../MyNet/data/* build/darknet/x64/data/

##Labels

In [52]:
#!cp build/darknet/x64/data/train/_darknet.labels build/darknet/x64/data/obj.names
!mv data/coco.names data/coco_old.names
!cp build/darknet/x64/data/obj.names data/coco.names
#!mv data/coco_old.names data/coco.names

## obj.data file

In [7]:
obj_data = """classes= 1
train  = build/darknet/x64/data/train.txt
valid  = build/darknet/x64/data/valid.txt
names = build/darknet/x64/data/obj.names
backup = build/darknet/x64/backup/
"""

file = """text_file = open("build/darknet/x64/data/obj.data", "w");text_file.write(obj_data);text_file.close()""" 

exec(file)
%pycat build/darknet/x64/data/obj.data

##Train file and valid file

In [15]:
!ls build/darknet/x64/data/train/*.jpg > build/darknet/x64/data/train.txt
!ls build/darknet/x64/data/valid/*.jpg > build/darknet/x64/data/valid.txt
!ls build/darknet/x64/data/test/*.jpg > build/darknet/x64/data/test.txt

#Starting net training

In [ ]:
!./darknet detector train build/darknet/x64/data/obj.data cfg/myown.cfg weights/darknet53.conv.74 -dont_show -map

#Resuming training

In [ ]:
!cp build/darknet/x64/backup/myown_last.weights weights/myown.weights
!./darknet detector train build/darknet/x64/data/obj.data cfg/myown.cfg weights/myown.weights -dont_show -map

#Copying best weights

In [64]:
!cp build/darknet/x64/backup/myown_best.weights weights/myown.weights

# Plotting the training accuracy history

In [ ]:
!./darknet detector map build/darknet/x64/data/obj.data cfg/myown.cfg weights/myown.weights -dont_show
from IPython.display import Image, display
display(Image('chart.png'))
#display(Image('chart_myown.png'))

#Example detection

In [ ]:
# yolov2
#!./darknet detect cfg/yolov2.cfg weights/yolov2.weights data/person.jpg

# yolov3-tiny
#!./darknet detect cfg/yolov3-tiny.cfg weights/yolov3-tiny.weights data/giraffe.jpg

# yolov3
#!./darknet detect cfg/yolov3.cfg weights/yolov3.weights data/dog.jpg

# myown
!./darknet detect cfg/myown.cfg weights/myown.weights build/darknet/x64/data/train/h22_jpg.rf.df8c90641790dcaf15663ed09e6b4ba5.jpg -dont_show -thresh 0.2 

# Checking net accuracy

In [ ]:
!./darknet detector test build/darknet/x64/data/obj.data cfg/myown.cfg weights/myown.weights -dont_show < build/darknet/x64/data/train.txt

#Displaying detections

In [ ]:
from IPython.display import Image, display
display(Image('predictions.jpg'))